In [20]:
import jieba
import os
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from wordcloud import WordCloud
from os import path
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.sklearn
import pyLDAvis
import joblib

In [2]:
data = pd.read_excel('clean_data.xlsx')
data

,target,userId,sellerId,timestamp,comment
0,1,127,1708,2019-10-01 00:57:45,订餐 半小时 不见 餐 催 挂 电话
1,1,1661,886,2019-10-01 00:42:07,太 不靠 谱 漏发 订单 菜 送错 早 送 小时
2,0,92758,10612,2019-10-01 00:51:18,超级 超级 喜欢 吃 每次 好开心 口味 建议 手套 戴 几双 吃 完 形象...
3,0,1639,1699,2019-10-03 00:01:56,谢谢 快递 哥哥 准时 辛苦
4,1,241839,36556,2019-10-01 00:07:56,差得 餐厅 上菜 慢 贵 餐厅 饿死 那种 面条 味道 一碗 ...
...,...,...,...,...,...
17948,0,34025,24833,2020-09-25 23:15:36,感觉 好像 发挥 稳定 几此 有好有坏 吃 日料 关注 三文鱼 中等 不...
17949,1,656,901,2020-09-25 23:17:49,订 九点 送到 十二点
17950,1,612,743,2020-09-27 23:36:59,差评 辣白菜 五花肉 石锅 拌 饭 超级 难吃 五花肉 硬 难吃 感觉 新鲜 ...
17951,1,146739,15687,2020-09-27 23:53:15,贵 好吃 老婆 几次 非要 一吃 真 难吃 蒸 没错 炒菜 油腻 菜 好...


In [12]:
data = data[data.target==0]
data.to_excel('./Q3/group_acclaim_data.xlsx')
data

,target,userId,sellerId,timestamp,comment
2,0,92758,10612,2019-10-01 00:51:18,超级 超级 喜欢 吃 每次 好开心 口味 建议 手套 戴 几双 吃 完 形象...
3,0,1639,1699,2019-10-03 00:01:56,谢谢 快递 哥哥 准时 辛苦
5,0,13116,20542,2019-10-03 00:38:01,超爱 美食 滴 小孩 大额 天堂 芋 贵人 宝珠 奶酪 每次 必去 分店
9,0,907,1544,2019-10-03 00:36:54,汤包 挺好吃 订
10,0,561,1684,2019-10-03 00:04:29,味道 不错 实惠
...,...,...,...,...,...
17944,0,120814,23801,2020-09-27 23:45:42,西单 新世界 这回 营业 中午 凯撒大帝 分量 面包 饮...
17945,0,163083,21216,2020-09-25 23:51:59,鸡皮 鸡皮 爱 盐 酥 鸡 下次 要尝下 地瓜 条 外卖 无缘 薯条 好...
17946,0,104843,21933,2020-09-25 23:29:57,地方 不错 吃饭 感觉 舒服 菜品 算不上 特别 可口 价钱 低 一点 ...
17947,0,19153,24467,2020-09-25 23:11:59,好吃 一口气 吃 生 煎 老板 舍得 放 白芝麻 满嘴 芝麻 太香 大爱...


In [23]:
counts = Counter(data['sellerId'])
print(max(counts.keys(), key=counts.get))
print(counts.most_common(1))

1041
[(1041, 54)]


In [25]:
data = data[data.sellerId==max(counts.keys(), key=counts.get)]
data.to_excel('./Q3/group_max_acclaim_data.xlsx')
data

,target,userId,sellerId,timestamp,comment
69,0,223,1041,2019-10-04 00:18:45,量 味道 不错 性价比 超高
115,0,1115,1041,2019-10-03 00:15:06,包装 用心 点赞
390,0,172,1041,2019-10-08 00:31:18,包装 漂亮
642,0,1996,1041,2019-10-14 01:39:49,包装 不错 味道 不错
772,0,1935,1041,2019-10-18 02:11:36,价格 实惠 值得 购买
799,0,1240,1041,2019-10-17 02:58:52,配送 员 很棒
927,0,1118,1041,2019-10-19 02:29:20,好吃 包装 干净 卫生
949,0,1266,1041,2019-10-20 02:21:40,服务 速度 专业
1013,0,944,1041,2019-10-23 02:02:29,实惠 吃不完
1168,0,317,1041,2019-10-24 03:05:28,真不错 价格便宜 实惠 分量 足 满意 一家


In [27]:
cut_text=''
for i in data[data['target']==0]['comment']:
    cut_text=cut_text+str(i)
cut_textlist=cut_text.split()
sorted(Counter(cut_textlist).items(),key=lambda x:x[1],reverse=True)[:10]

[('不错', 10),
 ('味道', 8),
 ('实惠', 5),
 ('速度', 5),
 ('服务', 5),
 ('特别', 4),
 ('小哥', 3),
 ('态度', 3),
 ('好吃', 3),
 ('辛苦', 3)]

In [30]:
plt.figure(figsize=(20,20),dpi=800)
cloud=WordCloud(font_path="./font.ttf",
                background_color='white',
                max_words=1000,
                scale=4)
word_cloud=cloud.generate_from_text(cut_text)
plt.imshow(word_cloud)
plt.axis('off')
plt.savefig('./Q3/max_sellerId_acclaim.jpg')
plt.show()

In [21]:
source_path = './Q3/group_max_acclaim_data.xlsx'

document_column_name = 'comment'

top_words_csv_path = './Q3/top-topic-words.xlsx'

predict_topic_csv_path = './Q3/document-distribution.xlsx'

html_path = './Q3/document-lda-visualization-top.html'

n_topics = 5

n_top_words = 10

 
 
def top_words_data_frame(model: LatentDirichletAllocation,tf_idf_vectorizer: TfidfVectorizer, n_top_words: int) -> pd.DataFrame:
    rows = []
    feature_names = tf_idf_vectorizer.get_feature_names()
    for topic in model.components_:
        top_words = [feature_names[i]
                     for i in topic.argsort()[:-n_top_words - 1:-1]]
        rows.append(top_words)
    columns = [f'topic {i+1}' for i in range(n_top_words)]
    df = pd.DataFrame(rows, columns=columns)
 
    return df
 
 
def predict_to_data_frame(model: LatentDirichletAllocation, X: np.ndarray) -> pd.DataFrame:
    matrix = model.transform(X)
    columns = [f'P(topic word {i+1})' for i in range(len(model.components_))]
    df = pd.DataFrame(matrix, columns=columns)
    return df
 
 
 
def predict_new_text(text_to_predict_list):
    lda = joblib.load('./Q3/lda-joblib.pkl')
    tf_idf_vectorizer = joblib.load('./Q3/tf_idf_vectorizer.pkl')
    cuts = []
    for text in text_to_predict_list:
        cut_text = [' '.join(jieba.lcut(text))]
        cuts += cut_text
    print(cuts)
    text_tf_idf = tf_idf_vectorizer.transform(cuts).toarray()
    print(text_tf_idf.shape)
    print(text_tf_idf)

    nonzero = np.nonzero(text_tf_idf)
    print(nonzero)
    print(np.array(nonzero).ndim)
    text_predict_df = predict_to_data_frame(lda, text_tf_idf)
    print('预测文本的主题概率分布情况:')
    print(text_predict_df)
if not os.path.exists(source_path):
    print(f'请用浏览器打开 {url} 并下载该文件(如果没有自动下载，则可以在浏览器中按键盘快捷键 ctrl s 来启动下载)')
    os.exit()
df = (
    pd.read_excel(source_path)
    .drop_duplicates()
    .rename(columns={
        document_column_name: 'text'
    }))
# 去重、去缺失、分词
df['cut'] = (
    df['text']
    .apply(lambda x: str(x))
    .apply(lambda x: re.sub(pattern, ' ', x))
    .apply(lambda x: " ".join(jieba.lcut(x)))
)

tf_idf_vectorizer = TfidfVectorizer()
tf_idf = tf_idf_vectorizer.fit_transform(df['cut'])
lda = LatentDirichletAllocation(
    n_components=n_topics,
    max_iter=50,
    learning_method='online',
    learning_offset=50,
    random_state=0)

lda.fit(tf_idf)

top_words_df = top_words_data_frame(lda, tf_idf_vectorizer, n_top_words)

top_words_df.to_csv(top_words_csv_path, encoding='utf-8-sig', index=None)

X = tf_idf.toarray()

predict_df = predict_to_data_frame(lda, X)

predict_df.to_csv(predict_topic_csv_path, encoding='utf-8-sig', index=None)


joblib.dump(lda, 'lda-joblib.pkl')
joblib.dump(tf_idf_vectorizer, 'tf_idf_vectorizer.pkl')

#使用 pyLDAvis 进行可视化
data = pyLDAvis.sklearn.prepare(lda, tf_idf, tf_idf_vectorizer)
pyLDAvis.save_html(data, html_path)
# 清屏
os.system('clear')
# 浏览器打开 html 文件以查看可视化结果
os.system(f'start {html_path}')
print('本次生成了文件：',
      top_words_csv_path,
      predict_topic_csv_path,
      html_path)

print(lda.perplexity(tf_idf))  # 收敛效果

本次生成了文件： ./Q3/top-topic-words.xlsx ./Q3/document-distribution.xlsx ./Q3/document-lda-visualization-top.html
784.9951166229798
